# Consolidate Data (1)

In this module, each data table with encoded encounters ranked by time of occurrence will be loaded. Conditions will also be loaded and all conditions occurring prior to pre-diabetes will be kept as features. The incidence of diabetes will be encoded as a binary response variable. All diabetes conditions will be dropped from downstream analysis because the incidence of diabetes implies that at some stage a prediabetic stage must have taken place (especially for Type 2 Diabetes Mellitus).

---
---

## Prep

### Load Libraries

In [1]:
import pandas as pd
import numpy as np

---

### Functions

#### `PATIENT` to index & explode

In [3]:
def patient_index_explode(tab):
    # Create a new column 'encounter' to differentiate between duplicate rows for each patient
    tab['encounter_n'] = tab.groupby('PATIENT').cumcount()+1
    tab = tab.pivot_table(index='PATIENT',
                          columns='encounter_n',
                          aggfunc='first')
    # Flatten the MultiIndex columns
    tab.columns = [f'{col[0]}_{col[1]}' for col in tab.columns]
    # Fill in nulls with 0
    tab.fillna(0, inplace=True)
    # Convert to uint32
    tab=tab.astype('uint32')
    sparse_dtype = pd.SparseDtype(np.uint32, fill_value=0)
    tab = tab.astype(sparse_dtype)
    return tab

---

### Read & Tidy

In [5]:
def read_n_tidy (tab_name_pkl):
    tab = pd.read_pickle(tab_name_pkl)
    tab.drop_duplicates (inplace=True)
    tab.fillna(0, inplace=True)
    cols2trans = list(tab.columns)
    for col in cols2trans:
        print(col)
        if col == 'PATIENT':
            continue
        else:
            tab[col] = tab[col].sparse.to_dense().astype(np.uint32)
    return tab

---
---

## Prepare tables for merging

Tables will be loaded and transformed such that for each patient there is a multitude of numerically encoded data in each column. The table will contain encounter codes with a parallel table containing corresponding encounter values. The encounter codes and the `encounters` table will be used to sort the data in a time-depending manner for both encounter codes and encounter values in order to be able to input a sequence of events (and their values) for RNN construction.

---

### Patients

In [ ]:
patients = pd.read_pickle('patients2.pkl')
print('patients:', patients.shape)

A total of a bit more than 1 million patients

#### Tidy up `patients`

##### Handle null values

In [ ]:
patients.isna().sum()

There are many null values for marital status. Let us explore the different categories for this feature.

In [ ]:
patients['MARITAL'].value_counts()

There are 2 categories for `MARITAL`:
- M (married)
- S (single)
  
  
These will be encoded in the following way:
- M (married) - 1
- S (single) - 0
- Null (single) - 0  
  
Note that the null values are lumped together with single as one does not know what the marital status of these patients is (likely single).

In [ ]:
patients['MARITAL'] = patients['MARITAL'].map({'M' : 1,
                                         'S' : 0,
                                         np.nan : 0
                                        })
# Sanity check:
print(patients['MARITAL'].value_counts())
print('Total nulls for "MARITAL":', patients['MARITAL'].isna().sum())

##### Numerically encode features

In [ ]:
category_count = {col : patients[col].value_counts() for col in patients.columns[1:]}
for v in category_count:
    print(v)
    print (category_count[v],'\n---------------\n')

###### `GENDER`

This feature will be encoded as follows:
- M (male) : 0
- F (female) : 1

In [ ]:
patients['GENDER'] = patients['GENDER'].map({'M' : 0, 'F' : 1})

##### `BIRTHDATE` - ***feature engineering***

`BIRTHDATE` will be used to calculate the time elapsed (in months) from birth until March 1, 2018 (the creation of the synthetic medical records dataset). This feature will be an integer. Months was selected here instead of days or years because it provides a good balance between two competing aims of accuracy and saving memory.

In [ ]:
# Declare the reference date:
reference_date = pd.to_datetime('2018-03-01')
# Calculate months since birth:
patients['m_since_birth'] = ((reference_date - patients['BIRTHDATE'])/np.timedelta64(1, 'M')).astype(int)
# Plot the result for visualization
patients['m_since_birth'].hist(bins=30)

Great. No outliers either.

In [ ]:
patients.drop('BIRTHDATE', axis=1, inplace=True)

#### `RACE` and `ETHNICITY`

`RACE` and `ETHNICITY` will simply be 1-hot-encoded.

In [ ]:
# 1-hot encode variables
patients_1hot = pd.get_dummies(patients, columns=['RACE', 'ETHNICITY'])
# Sanity check:
display(patients_1hot.head())

##### Downcast numbers

In order to save space, all numbers will be downcast. Specifically, all columns except `PATIENT` and `m_since_birth` will be assigned type `UInt8`

In [ ]:
cols2trans = list(patients_1hot.columns)
print(cols2trans)

In [ ]:
for col in cols2trans:
    print(col)
    if col == 'PATIENT':
        continue
    elif col == 'm_since_birth':
        patients_1hot[col] = patients_1hot[col].astype(np.uint16)
    else:
        patients_1hot[col] = patients_1hot[col].astype(np.uint8)

In [ ]:
# Check types:
print(patients_1hot.dtypes)

Great!  
Now I will check again for missing/null values.

In [ ]:
patients_1hot.isna().sum()

`patients` table is properly encoded and contains no nulls.

##### Duplicates

In [ ]:
# Check how many duplicates there are:
print(patients_1hot.duplicated().sum())

No duplicated rows so data can be properly saved.

In [ ]:
patients = patients_1hot
del patients_1hot
print('Dimensions of patients table --', patients.shape)

##### Index and save

In [ ]:
patients.set_index('PATIENT', inplace=True)

In [ ]:
patients = pd.read_pickle('patients2.pkl')
display(patients.head())

In [ ]:
# Save table:
patients.to_pickle('patients2.pkl')

In [ ]:
# Clear memory
del patients, col, cols2trans

---

### Allergies (encounters)

#### Read table

In [ ]:
allergies = pd.read_pickle('allergies_encounters.pkl')
print('allergies:', allergies.shape)

#### Tidy up `allergies`

##### Check for duplicates

In [ ]:
print(allergies.duplicated().sum())

##### Check for null values

In [ ]:
print(allergies.isna().sum())

##### Transform data type

First, take a look at the data type for each column.

In [ ]:
print(allergies.dtypes)

Each column is of sparse format and contains int64. This is an overkill. `np.uint32` will be used since all the values are positive integer codes for encounters.

In [ ]:
cols2trans = list(allergies.columns)
print(cols2trans)

In [ ]:
for col in cols2trans:
    print(col)
    if col == 'PATIENT':
        continue
    else:
        allergies[col] = allergies[col].sparse.to_dense().astype(np.uint32)

In [ ]:
# Sanity check:
print(allergies.dtypes)

Check how many of the patient id's (`PATIENT`) are duplicated.

In [ ]:
print('Duplicates for patients:', allergies['PATIENT'].duplicated().sum())

So mostly duplicated. The table will be arranged such that each row corresponds to a single patient and each column, to a single  encounter type with values corresponding to the encounter time rank.

In [ ]:
# Create a new column 'encounter' to differentiate between duplicate rows for each patient
allergies['encounter_n'] = allergies.groupby('PATIENT').cumcount()+1
# Sanity check:
display(allergies.head())

In [ ]:
allergies_pivot = allergies.pivot_table(index='PATIENT', 
                                        columns='encounter_n', 
                                        aggfunc='first')
# Sanity check:
display (allergies_pivot.head())

In [ ]:
# Flatten the MultiIndex columns
allergies_pivot.columns = [f'{col[0]}_{col[1]}' for col in allergies_pivot.columns]
# Sanity check:
display(allergies_pivot.head())

In [ ]:
print('dimensions for allergies:', allergies_pivot.shape)

In [ ]:
allergies = allergies_pivot
del allergies_pivot, col

The null values obtain after pivoting the table will be replaced with 0 indicating no encounter for the patient.

In [ ]:
allergies.fillna(0, inplace=True)
# Convert to uint32
allergies=allergies.astype('uint32')
# Sanity check:
display(allergies.head())

In [ ]:
# Convert the DataFrame to a SparseDataFrame
sparse_dtype = pd.SparseDtype(np.uint32, fill_value=0)
allergies = allergies.astype(sparse_dtype)
display(allergies.head())

In [ ]:
# Save table:
allergies.to_pickle('allergies_encounters.pkl')

In [ ]:
del allergies

---

### Careplans (encounters)

#### Read table

In [ ]:
careplans = pd.read_pickle('careplans_encounters.pkl')
print('careplans:', careplans.shape)

#### Tidy up `careplans`

##### Check for duplicates

In [ ]:
print(careplans.duplicated().sum())

Quite a lot of duplicated rows are present. Only the first of each duplicates will be kept for downstream analysis.

In [ ]:
careplans.drop_duplicates(inplace=True)

##### Check for null values

In [ ]:
print(careplans.isna().sum())
print(careplans.isna().sum().sum())

No null values.

##### Transform data type

First, take a look at the data type for each column.

In [ ]:
print(careplans.dtypes)

Each column is of sparse format and contains int64. This is an overkill. `np.uint32` will be used since all the values are positive integer codes for encounters.

In [ ]:
cols2trans = list(careplans.columns)
print(cols2trans)

In [ ]:
for col in cols2trans:
    print(col)
    if col == 'PATIENT':
        continue
    else:
        careplans[col] = careplans[col].sparse.to_dense().astype(np.uint32)

In [ ]:
# Sanity check:
print(careplans.dtypes)

Check how many of the patient id's (`PATIENT`) are duplicated.

In [ ]:
print('Duplicates for patients:', careplans['PATIENT'].duplicated().sum())

So mostly duplicated. The table will be arranged such that each row corresponds to a single patient and each column, to a single  encounter type with values corresponding to the encounter time rank.

In [ ]:
# Create a new column 'encounter' to differentiate between duplicate rows for each patient
careplans['encounter_n'] = careplans.groupby('PATIENT').cumcount()+1
# Sanity check:
display(careplans.head())

In [ ]:
careplans = careplans.pivot_table(index='PATIENT', 
                                        columns='encounter_n', 
                                        aggfunc='first')
# Sanity check:
display (careplans.head())

In [ ]:
# Flatten the MultiIndex columns
careplans.columns = [f'{col[0]}_{col[1]}' for col in careplans.columns]
# Sanity check:
display(careplans.head())

In [ ]:
print('dimensions for careplans:', careplans.shape)

Check for duplicated rows

In [ ]:
careplans.duplicated().sum()

The values will be converted to uint32 and null values will be replaced with 0 (no encounter for the careplan).

In [ ]:
careplans.fillna(0, inplace=True)
# Convert to uint32
careplans=careplans.astype('uint32')
# Sanity check:
display(careplans.head())

In [ ]:
# Convert the DataFrame to a SparseDataFrame
sparse_dtype = pd.SparseDtype(np.uint32, fill_value=0)
careplans = careplans.astype(sparse_dtype)

In [ ]:
display(careplans.head())

In [ ]:
# Save table:
careplans.to_pickle('careplans_encounters.pkl')

In [ ]:
del careplans

---

### Conditions (encounters)

#### Read and tidy table

In [ ]:
conditions = read_n_tidy ('conditions_encounters.pkl')
display(conditions.head())
print('Dimensions:', conditions.shape)

In [ ]:
conditions = patient_index_explode(conditions)

In [ ]:
display(conditions.head())
print('Dimensions:', conditions.shape)

In [ ]:
# Save table:
conditions.to_pickle('conditions_encounters.pkl')

In [ ]:
del conditions

---

### Immunizations (encounters)

#### Read and tidy table

In [ ]:
immunizations = read_n_tidy ('immunizations_encounters.pkl')
display(immunizations.head())
print('Dimensions:', immunizations.shape)

In [ ]:
immunizations = patient_index_explode(immunizations)

In [ ]:
display(immunizations.head())
print('Dimensions:', immunizations.shape)

In [ ]:
# Save table:
immunizations.to_pickle('immunizations_encounters.pkl')

In [ ]:
del immunizations

---

---
---
---

In [ ]:
encounters = pd.read_pickle('encounters1.pkl')
print(encounters.shape)

In [15]:
encounters.sort_values(by=['DATE','PATIENT'], inplace=True)
display(encounters.head(20))

,ID,DATE,PATIENT,CODE,DESCRIPTION,encounter_code
9535747,65954e04-00ca-47ef-bec8-317201ee59f4,1911-06-22,86d10918-697f-4bdd-8cb0-41ff0501dc4b,308646001,Death Certification,1
14031986,3c105f4b-e6f0-4bd7-a157-3f5df5772d63,1912-02-15,12d4436a-51b0-4d0d-962d-a7ea3b8508c4,308646001,Death Certification,2
12077159,ee6919b9-ffc0-43e4-8eff-59c720b720b6,1912-06-02,b934509e-5016-4a3b-a73f-452b9279a9e8,308646001,Death Certification,3
295927,8c0b4a8a-c0e7-4a6a-af21-e1a551bfc0f5,1913-03-29,f129ddbd-f2d6-441a-94b2-fbb812b2539a,308646001,Death Certification,4
8351882,54540535-807f-4b69-b36d-9837a9c5989f,1913-06-29,955e0259-a9d7-4805-9b23-d612c8d922a5,308646001,Death Certification,5
12510007,aec2b8ee-4e3a-4b14-9244-22b9864a54b9,1913-09-27,265e186c-0479-4eef-a611-aa3c76496993,308646001,Death Certification,6
7655706,6b4b50df-bf89-4853-89f7-11de030a0597,1913-12-22,e0b60100-4906-4b28-aa0e-65bbd45f3b3e,308646001,Death Certification,7
5707461,7bcf21e4-42a2-4037-9c9d-676facc8fea4,1914-04-15,689b8db9-5726-430b-8be1-09108161e14e,308646001,Death Certification,8
70878,4e92b4de-3225-425f-b1b8-3bed71c30c63,1915-01-30,1cde28d1-ca7a-49f2-b72a-d39554f3dd4d,308646001,Death Certification,9
12890807,5fd2dd84-4abf-426f-9980-5122babf7b56,1915-03-28,bfbb7183-da46-4da7-a467-f618e90561cf,308646001,Death Certification,10


In [7]:
# Create a new column 'encounter' to differentiate between duplicate rows for each patient
encounters['encounter_n'] = encounters.groupby('PATIENT').cumcount()+1
display(encounters.tail())

,ID,DATE,PATIENT,CODE,DESCRIPTION,encounter_code,encounter_n
9535747,65954e04-00ca-47ef-bec8-317201ee59f4,1911-06-22,86d10918-697f-4bdd-8cb0-41ff0501dc4b,308646001,Death Certification,1,1
14031986,3c105f4b-e6f0-4bd7-a157-3f5df5772d63,1912-02-15,12d4436a-51b0-4d0d-962d-a7ea3b8508c4,308646001,Death Certification,2,1
12077159,ee6919b9-ffc0-43e4-8eff-59c720b720b6,1912-06-02,b934509e-5016-4a3b-a73f-452b9279a9e8,308646001,Death Certification,3,1
295927,8c0b4a8a-c0e7-4a6a-af21-e1a551bfc0f5,1913-03-29,f129ddbd-f2d6-441a-94b2-fbb812b2539a,308646001,Death Certification,4,1
8351882,54540535-807f-4b69-b36d-9837a9c5989f,1913-06-29,955e0259-a9d7-4805-9b23-d612c8d922a5,308646001,Death Certification,5,1


In [9]:
del encounters

>At this point, the data becomes way too large. A novel approach will be adopted:
>1. Obtain batches of data.
>2. Transform into a squence of events with values.
>3. Test/train split.
>4. Train a separate RNN for each batch.
>5. Use outputs from the individual RNN batches to train a meta model.
>6. Optimize meta model using a validation subset.
>7. Assess meta model on a test subset (the last batch)

---